# Run AROSICS Geometric Calibration on Planet Images
## TODO:
- possible to get AROSICS to fail quietly?
- delete masked file after running AROSICS

## Import Libraries

In [ ]:
import xarray as xr
import rioxarray as rxr
from geoarray import GeoArray
from arosics import COREG
from arosics import COREG_LOCAL
import os
import re
from pprint import pprint

## Define Functions

In [ ]:
def makemydir(dir_path):
    try:
        os.makedirs(dir_path)
    except OSError:
        pass
    os.chdir(dir_path) # this changes the working directory - necessary?

## Define Directories and Load Data

In [ ]:
# Read in data
ref_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/sentinel_data/yamal_gydan_polygons'
base_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/planet_data/yamal_gydan_polygons'
ref_files = []
for root, subdirs, files in os.walk(ref_dir):
    for file in files:
        if re.match('.*tif$', file):
            ref_files.append(os.path.join(root, file))
        
tgt_files = []
for root, subdirs, files in os.walk(base_dir):
    for file in files:
        if re.match('.*SR.*tif$', file):
            tgt_files.append(os.path.join(root, file))
            
tgt_files = sorted(tgt_files)
tgt_files = [file for file in tgt_files if not re.match('.*/2017/.*tif$', file)]
            
udm_files = []
for root, subdirs, files in os.walk(base_dir):
    for file in files:
        if re.match('.*udm2.*tif$', file):
            udm_files.append(os.path.join(root, file))
            
udm_files = sorted(udm_files)
udm_files = [file for file in udm_files if not re.match('.*/2017/.*tif$', file)]

pprint(len(tgt_files))
pprint(len(udm_files))

mask_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/automated_download/arosics_output/masked_files/'
makemydir(mask_dir)
out_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/automated_download/arosics_output/calibrated_files/'
makemydir(out_dir)
out_fig_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/automated_download/arosics_output/figures/'
makemydir(out_fig_dir)

In [ ]:
tgt_files_subset = [filepath for filepath in tgt_files if re.match('.*/polygon_id_(5|6|7|8|9)/.*tif$', filepath)]
print(len(tgt_files_subset))
udm_files_subset = [filepath for filepath in udm_files if re.match('.*/polygon_id_(5|6|7|8|9)/.*tif$', filepath)]
print(len(udm_files_subset))

In [ ]:
start_idx = 0

## Run AROSICS

In [ ]:
for tgt_file, udm_file in zip(tgt_files_subset[start_idx:-1], udm_files_subset[start_idx:-1]):
    
    # get crs of tgt file to determine correct ref file
    crs = str(rxr.open_rasterio(tgt_file).rio.crs).split(':')[1][3:5]
    
    # get correct ref file
    polygon_id = tgt_file.split('/')[10].split('_')[2]
    ref = [file for file in ref_files if re.match('.*' + crs + 'N_' + polygon_id + '.tif$', file)][0]
    
    # Mask imagery using clear mask
    tgt = rxr.open_rasterio(tgt_file)
    udm = rxr.open_rasterio(udm_file).sel(band = 1)
    tgt = tgt.where(tgt != 0)
    tgt = tgt.where(udm != 0)
    
    # save masked file
    tgt.rio.to_raster(os.path.join(mask_dir, polygon_id + '_' + tgt_file.split('/')[-1][0:-4] + '_masked.tif'))
    
    # Apply AROSICS correction from red band to all bands
    out_path = os.path.join(out_dir, polygon_id + '_' + tgt_file.split('/')[-1][0:-4] + '_arosics.tif')
    out_fig_path = os.path.join(out_fig_dir, polygon_id + '_' + tgt_file.split('/')[-1][0:-4] + '_coreg_points.tif')
    kwargs = {
        'grid_res'     : 200, # can make this larger to improve run time
        'window_size'  : (200, 200), # A larger window size could slow things down, but too small might lead to less accurate alignment
        'q'            : False,
        'r_b4match'    : 3,
        's_b4match'    : 3,
        'path_out'     : out_path,
        'fmt_out'      : 'GTiff',
        'CPUs'         : 19
    }
    tgt_cal = COREG_LOCAL(ref, tgt_file, **kwargs)
    tgt_cal.correct_shifts(min_points_local_corr = 4)
    tgt_cal.view_CoRegPoints(savefigPath = out_fig_path)
    

In [ ]:
# if the AROSICS processing loop stops running, and it's due to no tie points (too small, cloudy, etc.),
# use this to skip the offending image and restart at the next image
start_idx = [x for x in range(len(tgt_files_subset)) if tgt_files_subset[x] == tgt_file][0]+1
start_idx